In [16]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from scipy import stats

In [17]:
np.random.seed(42)

digits = load_digits()  #loading digits dataset
data = scale(digits.data)  #scaling the data
data =  np.asmatrix(data)  #converting data into a numpy matrix

np.random.shuffle(data)  #shuffling the data randomly

n_rows, n_features = data.shape  #Getting the number of samples and features from data
n_digits = len(np.unique(digits.target))  #counting number of unique digits target
labels = digits.target  #assigning the target

print("Number of Digits:",n_digits)
print("Number of Samples:",n_rows)
print("Number of features:",n_features)

Number of Digits: 10
Number of Samples: 1797
Number of features: 64


In [18]:
#Fitting the data to kmeans algorithm

kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(data)

KMeans(n_clusters=10)

In [19]:
InitNumber = 100  #number is chosen as per computational complexity of kmeans

q = 20 #number of micro-clusters

initial_data = data[0:InitNumber,:]  #creating a subset of dataset of 100 samples as starter dataset
stream_data = data[InitNumber:len(data),:]  #The rest of the data is taken as data which is to be streamed
initial_data

matrix([[ 0.        , -0.33501649, -1.09493684, ...,  1.22664392,
          0.71700059, -0.19600752],
        [ 0.        , -0.33501649,  1.21914597, ..., -0.97712664,
         -0.5056698 , -0.19600752],
        [ 0.        , -0.33501649,  0.58803248, ...,  0.20951905,
         -0.5056698 , -0.19600752],
        ...,
        [ 0.        , -0.33501649, -1.09493684, ..., -0.29904339,
         -0.5056698 , -0.19600752],
        [ 0.        , -0.33501649, -0.88456568, ..., -1.14664746,
         -0.5056698 , -0.19600752],
        [ 0.        , -0.33501649, -0.46382335, ..., -1.14664746,
         -0.5056698 , -0.19600752]])

In [20]:
#Assigning the initial clusters using kmeans

kmeans = KMeans(init='k-means++', n_clusters=q).fit(initial_data)
initial_clusters = kmeans.cluster_centers_


In [21]:
micro_clusters = [None]*q

for i in range(0,len(initial_data)):
    
    starter_point = np.array(initial_data[i])  #Taking a data value from the dataset
    euc_dist = [ np.linalg.norm(starter_point - cluster) for cluster in initial_clusters ]  #Calculating the euclidean distance from data point to cluster centers
    cluster_id = np.argmin(euc_dist)  #getting minimum euclidean distance cluster
    
    CF2x = np.square(starter_point)  #square of data values
    CF1x = starter_point  #data values
    CF2t = np.square(i)  #square of time stamp
    CF1t = i  #time stamp
   
    cluster_tuple = np.array([CF2x,CF1x,CF2t,CF1t, 1])  #Adding the values to make a cluster tuple
    
    if (micro_clusters[cluster_id] == None):  # If cluster tuple does not belong to any micro-cluster
        micro_clusters[cluster_id] = cluster_tuple  #Add to a micro cluster
    else:
        micro_clusters[cluster_id] = micro_clusters[cluster_id] + cluster_tuple  #If already belongs, add to the existing tuple

ipykernel_launcher:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
ipykernel_launcher:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [22]:
print (micro_clusters)  #Displaying the micro clusters

74,  1.86922551, -1.33957027, -0.34266553, -0.10013918,
                5.59215002,  2.37891143, -1.60918638,  2.96220646, -0.28179973,
                1.91205488, -0.14169715,  0.        ,  7.75467225,  3.00467325,
                2.35925871,  2.37618697,  2.17548411,  4.60145568,  0.        ,
               -0.18403101,  4.10895793,  2.34933825,  4.08652509,  0.79633893,
               -2.75898787, -2.16385632, -0.26622485, -0.10629979, -0.637878  ,
               -1.68722207,  3.70918176, -3.25412891, -4.35783455, -2.27230743,
               -0.62935538, -0.07078938, -0.89724404, -0.13187786,  1.996784  ,
               -6.16866101, -3.43994237, -1.51700941, -0.58802256]])           ,
       15274, 184, 3], dtype=object), array([array([[0.00000000e+00, 7.85652327e-01, 8.39220685e+00, 2.08261082e+01,
               3.18449230e+00, 4.22579189e+00, 1.17511584e+00, 1.09415119e-01,
               2.44311043e-02, 2.72571292e+00, 1.93742627e+01, 1.23351656e+01,
               3.58116685e+0

In [23]:
#Calculating the squared distance of data points from the cluster centers

def calcSSQ(points, centroids, horizon):
    SSQ = 0
    for point in points[(-1*horizon):]:
        euc_dist = [ np.linalg.norm(point - cluster) for cluster in initial_clusters ]
        SSQ += np.square(min(euc_dist))
    print ("SSQ: " + str(SSQ))
    
    return SSQ

In [24]:
#Calculates the rmse deviation between square of arrival times and sum of squared arrival times
def RMSE(micro_clusters,cluster_id,n):
    squared_sum = np.square(micro_clusters[cluster_id][1])
    sum_of_squared = micro_clusters[cluster_id][0]

    RMSD = np.sqrt(np.abs(sum_of_squared - (squared_sum / n)))
                
    return RMSD

In [25]:
#Finding relevance timestamp as time of arrival of (2*nth) percentile of points in a cluster

def oldest_updated_cluster(mean_timestamp,Z,standard_deviation_timestamp):
    relevance_stamp = mean_timestamp + Z * standard_deviation_timestamp
        
    least_recent_cluster = np.argmin(relevance_stamp)  #Taking minimum to find the cluster which was least recently updated
    
    return relevance_stamp[least_recent_cluster],least_recent_cluster;

In [26]:
#Function to merge two micro clusters which are closest to one another

def merge_cluster(micro_clusters,Xik,timestep):
    minA_id = -1  #Assigning initial default value
    minB_id = -1
    min_dist = float("inf")
    for a in range (0, len(micro_clusters)):
        for b in range (a + 1, len(micro_clusters)):
            
            #Calculating distance between two clusters
            
            d = np.linalg.norm((micro_clusters[b][1] / micro_clusters[b][4]) - \
                               (micro_clusters[a][1] / micro_clusters[a][4]))
            
            #Setting the id values of clusters
            if d < min_dist:
                minA_id = a
                minB_id = b
                min_dist = d
            
    micro_clusters[minA_id] = micro_clusters[minA_id] + micro_clusters[minB_id]  #Merging two clusters

    micro_clusters[minB_id] = np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])  #Creating a new cluster
    print ("merged cluster")

In [27]:
def CluStream(record, timestep, micro_clusters, t = 2, r = 2, m = 10, delta = 10):
    
    Xik = np.array(record)  #Incoming data points
   
    euc_dist = [ np.linalg.norm(Xik - (cluster[1] / cluster[4])) for cluster in micro_clusters ]  #Distance between the incoming data points and cluster centers
    
    dist_sorted = np.argsort(euc_dist)  #Sorting the distances
    
    cluster = dist_sorted[0]  #Getting minimum value to assign datapoint to closest cluster
    
    i = 0
    while True:
        cluster_id = dist_sorted[i]
        n = micro_clusters[cluster_id][4]

        if n > 1:  #For defining maximal boundary only for clusters having more than 1 point
            
            S = RMSE(micro_clusters,cluster_id,n)  #Returns rms deviation
            
            maximal_boundary = np.linalg.norm(S) * t  #Defining maximum boundary as a factor of t of rms deviation
            
            if i > 0:
                maximal_boundary *= r
            break
            
        i += 1  #Finding the next closest cluster
        
    if dist[cluster] <= maximal_boundary: #If the data point falls within the maximum boundary of the micro-cluster
        
        #data point is added to the micro-cluster
        micro_clusters[cluster] = micro_clusters[cluster] + \
                                    np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])
        print ("add to cluster")
    else: 
        
        #create a new micro-cluster  
        mean_timestamp = [ (cluster[3] / cluster[4]) for cluster in micro_clusters ]  #Calculating mean of time stamp
        
        #calculating standard deviation of timestamp
        standard_deviation_timestamp = [ np.sqrt((cluster[2] / cluster[4]) - np.square((cluster[3] / cluster[4]))) \
   for cluster in micro_clusters ]  # stamp
        
        #assinging (2*nth) percentile of data points of a micro-cluster to Z
        Z = []
        for i in range(0, len(micro_clusters)):
            mc = m
            if mc > micro_clusters[i][4]:
                mc = micro_clusters[i][4]
                
            Z.append(m / (2 * micro_clusters[i][4]))
        
        Z = np.array(Z)
        
        #Returns the relevance stamp and the least recent cluster
        rs,lrc = oldest_updated_cluster(mean_timestamp,Z,standard_deviation_timestamp)  
        
        if rs < delta: #Checking whether relevance stamp is lesser than the threshold value defined
            
            micro_clusters[lrc] = np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])  #If value is less, delete the cluster and create a new cluster
            print ("eliminated cluster")
        
        else: 
            #merge the two micro-clusters which are closest to one another
            merge_cluster(micro_clusters,Xik,timestep)
            
        
    return micro_clusters

In [28]:
SSQ_kmeans = [] 
for i in range(0, len(stream_data)):
    SSQ_kmeans.append(calcSSQ((stream_data[0:(i+1)]), micro_clusters,100))

9.607914125698
SSQ: 5360.046749615387
SSQ: 5331.614897373938
SSQ: 5325.603901583282
SSQ: 5355.648349872624
SSQ: 5346.4229914529105
SSQ: 5324.602290731105
SSQ: 5311.544908385597
SSQ: 5307.876928771271
SSQ: 5310.078888126658
SSQ: 5314.182037308489
SSQ: 5331.513318290794
SSQ: 5285.5863440286175
SSQ: 5292.872330822155
SSQ: 5322.127345841702
SSQ: 5309.877595875197
SSQ: 5303.980986316741
SSQ: 5308.184028629067
SSQ: 5308.73113643882
SSQ: 5330.865024270146
SSQ: 5316.118216249785
SSQ: 5316.0003371052835
SSQ: 5323.482877097761
SSQ: 5324.898038534003
SSQ: 5346.710882788639
SSQ: 5334.82243578465
SSQ: 5347.000261041026
SSQ: 5355.348769599653
SSQ: 5344.370703351026
SSQ: 5342.778499853681
SSQ: 5337.314342388847
SSQ: 5361.904355198474
SSQ: 5368.092261437673
SSQ: 5358.222113186038
SSQ: 5359.699820201745
SSQ: 5326.070295636262
SSQ: 5314.908611276792
SSQ: 5213.643534162251
SSQ: 5196.448767619864
SSQ: 5171.772979302006
SSQ: 5190.07567058298
SSQ: 5161.165988455477
SSQ: 5157.669047282176
SSQ: 5120.459533077